In [ ]:
import matplotlib.pyplot as plt
import torch

from upper.geometry import compute_area
from upper.network import SofaNet

torch.set_default_dtype(torch.float64)
plt.rcParams['text.usetex'] = True

In [ ]:
# load model
model = SofaNet([128, 128, 128])
model.load_state_dict(torch.load("figs/data/upper_model.pt"))

# compute area
a1 = 0.283794109  # asin(7/25)
a2 = 0.532504098  # asin(33/65)
a3 = 0.781214087  # asin(119/169)
a4 = 1.03829223  # asin(56/65)
a5 = 1.28700222  # asin(24/25)
alpha = torch.tensor([a1, a2, a3, a4, a5])
u1, u2 = model.forward(alpha)
area = compute_area(alpha, torch.pi / 2, torch.pi / 2, u1, u2, n_areas=10000)
print(area.item())

In [ ]:
# the landscapes of `area` (Jaehoon)

# NOTE: landscape with respect to u1 and u2 instead of network weights

u1 and u2 are intermediate nodes in the original model. We create a new NN, making u1 and u2 into network weights so current libraries for Jacobian and Hessian can work.

In [ ]:
nu = u1.shape[0]


class U12Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(1, nu * 2, bias=False)
        self.fc.weight.data[:nu] = u1.unsqueeze(1)[:]
        self.fc.weight.data[nu:] = u2.unsqueeze(1)[:]

    def forward(self):
        x = torch.ones((1, 1))  # input must be [[1]]
        x = self.fc(x)
        u1_, u2_ = x.split(nu, dim=1)
        return u1_.squeeze(0), u2_.squeeze(0)

In [ ]:
fake_model = U12Net()
u1_net, u2_net = fake_model.forward()
print(u1_net.shape)

In [ ]:
# check they are same
print(u1)
print(u1_net)
print(fake_model.fc.weight[:nu])

print(u2)
print(u2_net)
print(fake_model.fc.weight[nu:])

### Now, we can use `fake_model` to plot the landscapes of `area`

In [ ]:
area = compute_area(alpha, torch.pi / 2, torch.pi / 2, u1_net, u2_net, n_areas=10000)